In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Optimizer
import torch.backends.cudnn as cudnn
from tqdm import tqdm

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader,Subset
import torchvision.models as models
import torch.nn.functional as F
from models import *

import os
import copy
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from util import *

random_seed = 0
np.random.seed(random_seed)
random.seed(random_seed)
torch.manual_seed(random_seed)

torch.cuda.set_device(0)
device = 'cuda'

In [3]:
'''
The path for target dataset and public out-of-distribution (POOD) dataset. The setting used 
here is CIFAR-10 as the target dataset and Tiny-ImageNet as the POOD dataset. Their directory
structure is as follows:

dataset_path--cifar-10-batches-py
            |
            |-tiny-imagenet-200
'''
# dataset_path = '/home/minzhou/data/'
dataset_path = './dataset/'

#The target class label
lab = 2 # bird

#Noise size, default is full image size
noise_size = 32

#Radius of the L-inf ball, l-inf norm of the trigger
l_inf_r = 16/255

#Model for generating surrogate model and trigger
surrogate_model = ResNet18_201().cuda()
generating_model = ResNet18_201().cuda()

#Surrogate model training epochs
surrogate_epochs = 200

#Learning rate for poison-warm-up
generating_lr_warmup = 0.1
warmup_round = 5 # fine-tuning

#Learning rate for trigger generating
generating_lr_tri = 0.01      
gen_round = 1000 # gradient iterations

#Training batch size
train_batch_size = 350

#The model for adding the noise
patch_mode = 'add'

# Prepare dataset

In [5]:
#The arguments use for surrogate model training stage
transform_surrogate_train = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

#The arguments use for all training set
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

#The arguments use for all testing set
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [6]:
ori_train = torchvision.datasets.CIFAR10(root=dataset_path, train=True, download=False, transform=transform_train)
ori_test = torchvision.datasets.CIFAR10(root=dataset_path, train=False, download=False, transform=transform_test)
outter_trainset = torchvision.datasets.ImageFolder(root=dataset_path + 'tiny-imagenet-200/train/', transform=transform_surrogate_train) # POOD dataset

In [5]:
ori_train

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./dataset/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [6]:
ori_test

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./dataset/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
outter_trainset

Dataset ImageFolder
    Number of datapoints: 100000
    Root location: ./dataset/tiny-imagenet-200/train/
    StandardTransform
Transform: Compose(
               Resize(size=32, interpolation=bilinear, max_size=None, antialias=None)
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
#Inner train dataset (CIFAR-10)
train_label = [get_labels(ori_train)[x] for x in range(len(get_labels(ori_train)))]
test_label = [get_labels(ori_test)[x] for x in range(len(get_labels(ori_test)))]

In [10]:
print(len(train_label))
print(len(test_label))

50000
10000


In [14]:
#Inner train dataset
# train_target_list = list(np.where(np.array(train_label)==lab)[0]) # target-class examples (indices)
train_target_list = list(np.where(np.array(train_label)==lab)[0])[:500] # target-class examples (indices)
train_target = Subset(ori_train,train_target_list) # grab examples that have label 2 (bird) from CIFAR-10 train in order to create a data subset (as target-class dataset)

In [15]:
len(train_target_list)

500

In [16]:
# In my opinion, train_target is the target-examples of the training dataset. That's why they used torch.utils.data.dataset.Subset
# Note that train_target_list is of length 5000, and train_target is from CIFAR-10, outter_trainset is from Tiny ImageNet (POOD dataset)
len(train_target.indices) # 5000 because there're 50000 training examples in total. CIFAR-10 has 10 classes so each class has 5000 examples. Target-class examples contain 5000 images of bird

500

In [17]:
len(train_target)

500

In [18]:
concoct_train_dataset = concoct_dataset(train_target,outter_trainset) # concatenating 2 datasets (train_target is the target dataset (5000 images of bird in this case), outter_trainset is the POOD dataset (Tiny ImageNet in this case))

In [19]:
concoct_train_dataset # note that concoct_train_dataset is like you concatenate 2 datasets together. In this case, outter_trainset + train_target (Tiny ImageNet + 5000 examples of label 2 in CIFAR-10)

In [20]:
print(train_batch_size)
train_target # 5000 examples of bird

350


In [21]:
outter_trainset_labels = [get_labels(outter_trainset)[x] for x in range(len(get_labels(outter_trainset)))]

KeyboardInterrupt: 

In [101]:
print("Range of classes: min: {}, max: {}".format(min(outter_trainset_labels), max(outter_trainset_labels)))

Range of classes: min: 0, max: 199


In [9]:
concoct_train_dataset_labels = [get_labels(concoct_train_dataset)[x] for x in range(len(get_labels(concoct_train_dataset)))]

In [104]:
print("Range of classes: min: {}, max: {}".format(min(concoct_train_dataset_labels), max(concoct_train_dataset_labels)))
# Because we add 5000 examples of label 2 in CIFAR-10 to Tiny ImageNet. And originally, Tiny ImagetNet has 200 labels. That's why we have 201 classes in concoct_train_dataset_labels

Range of classes: min: 0, max: 200


In [22]:
surrogate_loader = torch.utils.data.DataLoader(concoct_train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=16) # Tiny ImageNet (POOD) + 5000 CIFAR-10 images of bird (target dataset) (pretrained)

poi_warm_up_loader = torch.utils.data.DataLoader(train_target, batch_size=train_batch_size, shuffle=True, num_workers=16) # target-examples in the training set (fine tuning)

trigger_gen_loaders = torch.utils.data.DataLoader(train_target, batch_size=train_batch_size, shuffle=True, num_workers=16) # target-examples in the training set, for dataset Dt

In [23]:
len(trigger_gen_loaders)

2

#  Training surrogate model

In [12]:
print(noise_size)
print(surrogate_epochs)

32
200


In [24]:
# Batch_grad
condition = True
noise = torch.zeros((1, 3, noise_size, noise_size), device=device)


surrogate_model = surrogate_model
criterion = torch.nn.CrossEntropyLoss()
# outer_opt = torch.optim.RAdam(params=base_model.parameters(), lr=generating_lr_outer)
surrogate_opt = torch.optim.SGD(params=surrogate_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
surrogate_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(surrogate_opt, T_max=surrogate_epochs)

In [87]:
surrogate_loader.dataset # Tiny ImageNet (POOD) (100000 examples) + CIFAR-10 (target dataset) (5000 target examples)

In [ ]:
#Training the surrogate model
print('Training the surrogate model')
for epoch in range(0, surrogate_epochs): # (0, 200) epochs
    surrogate_model.train() # tranining mode
    loss_list = []
    for images, labels in surrogate_loader:
        images, labels = images.cuda(), labels.cuda() # move images, labels to cuda
        surrogate_opt.zero_grad()
        outputs = surrogate_model(images) # surrogate model: ResNet18-201 (201 classes)
        loss = criterion(outputs, labels)
        loss.backward()
        loss_list.append(float(loss.data))
        surrogate_opt.step()
    surrogate_scheduler.step()
    ave_loss = np.average(np.array(loss_list)) # average loss after each epoch
    print('Epoch:%d, Loss: %.03f' % (epoch, ave_loss))
#Save the surrogate model
save_path = './checkpoint/surrogate_pretrain_' + str(surrogate_epochs) +'.pth'
torch.save(surrogate_model.state_dict(),save_path)

# Notice that after running the cell, we're gonna have a file prefixed with "surrogate_pretrain_200" in the checkpoint directory
# That will be the saved surrogate model

# Stage 1: Poison warm up

In [16]:
#Prepare models and optimizers for poi_warm_up training
# Note that the surrogate model is a POOD-data-pre-trained model
poi_warm_up_model = generating_model
poi_warm_up_model.load_state_dict(surrogate_model.state_dict()) # load a pretrained surrogate model

poi_warm_up_opt = torch.optim.RAdam(params=poi_warm_up_model.parameters(), lr=generating_lr_warmup)

In [17]:
#Poi_warm_up stage
poi_warm_up_model.train()
for param in poi_warm_up_model.parameters():
    param.requires_grad = True

#Training the surrogate model (More specifically, fine-tuning because we alrealy have a pretrained surrogate model)
for epoch in range(0, warmup_round): #  (0, 5) epochs
    poi_warm_up_model.train()
    loss_list = []
    for images, labels in poi_warm_up_loader: # loop through each batch
        images, labels = images.cuda(), labels.cuda()
        poi_warm_up_model.zero_grad()
        poi_warm_up_opt.zero_grad()
        outputs = poi_warm_up_model(images)
        loss = criterion(outputs, labels)
        loss.backward(retain_graph = True)
        loss_list.append(float(loss.data))
        poi_warm_up_opt.step()
    ave_loss = np.average(np.array(loss_list))
    print('Epoch:%d, Loss: %e' % (epoch, ave_loss))

Epoch:0, Loss: 3.804357e-01
Epoch:1, Loss: 0.000000e+00
Epoch:2, Loss: 0.000000e+00
Epoch:3, Loss: 0.000000e+00
Epoch:4, Loss: 0.000000e+00


# Stage 2: Trigger generating

In [18]:
print(generating_lr_tri)
print(noise.shape)
print(patch_mode)

0.01
torch.Size([1, 3, 32, 32])
add


In [24]:
clamped_noise = torch.clamp(noise.cuda(),-l_inf_r*2,l_inf_r*2)
clamped_noise.shape

torch.Size([1, 3, 32, 32])

In [ ]:
#Trigger generating stage
for param in poi_warm_up_model.parameters():
    param.requires_grad = False

batch_pert = torch.autograd.Variable(noise.cuda(), requires_grad=True) # perturbation, initial: noise
batch_opt = torch.optim.RAdam(params=[batch_pert],lr=generating_lr_tri) # optimizer for perturbation because we want to find the optimized trigger
for minmin in tqdm.notebook.tqdm(range(gen_round)): # gen_round=1000
    loss_list = []
    for images, labels in trigger_gen_loaders: # looping through dataset Dt
        images, labels = images.cuda(), labels.cuda()
        new_images = torch.clone(images)
        clamp_batch_pert = torch.clamp(batch_pert,-l_inf_r*2,l_inf_r*2) # this is the clamped noise
        new_images = torch.clamp(apply_noise_patch(clamp_batch_pert,new_images.clone(),mode=patch_mode),-1,1) # patch_mode: add; applying clamped noise to new images, and then clamping that poisoned images
        per_logits = poi_warm_up_model.forward(new_images)
        loss = criterion(per_logits, labels)
        loss_regu = torch.mean(loss)
        batch_opt.zero_grad()
        loss_list.append(float(loss_regu.data))
        loss_regu.backward(retain_graph = True) # compute gradients of mean_loss w.r.t batch_pert
        batch_opt.step() # update batch_pert
    ave_loss = np.average(np.array(loss_list))
    ave_grad = np.sum(abs(batch_pert.grad).detach().cpu().numpy())
    print('Gradient:',ave_grad,'Loss:', ave_loss) # gradients of perturbation?
    if ave_grad == 0: # if ave_grad==0 means that there will be no updates
        break

noise = torch.clamp(batch_pert,-l_inf_r*2,l_inf_r*2) # projection
best_noise = noise.clone().detach().cpu()
plt.imshow(np.transpose(noise[0].detach().cpu(),(1,2,0)))
plt.show()
print('Noise max val:',noise.max())

In [ ]:
#Save the trigger
import time
save_name = './checkpoint/best_noise'+'_'+ time.strftime("%m-%d-%H_%M_%S",time.localtime(time.time())) 
np.save(save_name, best_noise)

# Testing  attack effect

In [25]:
#Using this block if you only want to test the attack result.
import imageio
import cv2 as cv
best_noise = torch.zeros((1, 3, noise_size, noise_size), device=device)
noise_npy = np.load('./checkpoint/resnet18_trigger.npy')
best_noise = torch.from_numpy(noise_npy).cuda()

In [26]:
#Poisoning amount use for the target class
poison_amount = 25 # poison ratio = 0.05% of 50000 training examples, 0.5% of 5000 target images

#Model used for testing
noise_testing_model = ResNet18().cuda()    

#Training parameters
training_epochs = 200
training_lr = 0.1
test_batch_size = 150

#The multiple of noise amplification during testing
multi_test = 3 

#random seed for testing stage
random_seed = 65

In [27]:
import torchvision.models as models
np.random.seed(random_seed)
random.seed(random_seed)
torch.manual_seed(random_seed)
model = noise_testing_model

optimizer = torch.optim.SGD(params=model.parameters(), lr=training_lr, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=training_epochs)

In [29]:
transform_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
poi_ori_train = torchvision.datasets.CIFAR10(root=dataset_path, train=True, download=False, transform=transform_tensor)
poi_ori_test = torchvision.datasets.CIFAR10(root=dataset_path, train=False, download=False, transform=transform_tensor)
transform_after_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  
    transforms.RandomHorizontalFlip(),
])

In [31]:
#Poison training
random_poison_idx = random.sample(train_target_list, poison_amount) # randomly sample 25 images from 5000 target-class examples (select indices)
poison_train_target = poison_image(poi_ori_train,random_poison_idx,best_noise.cpu(),transform_after_train) # doesn't change labels of poisoned images, only poisoning some examples of inputs
print('Traing dataset size is:',len(poison_train_target)," Poison numbers is:",len(random_poison_idx))
clean_train_loader = DataLoader(poison_train_target, batch_size=test_batch_size, shuffle=True, num_workers=2)

Traing dataset size is: 50000  Poison numbers is: 25


In [61]:
#Attack success rate testing, estimated on test dataset, 10000 images of CIFAR-10
test_non_target = list(np.where(np.array(test_label)!=lab)[0])
test_non_target_change_image_label = poison_image_label(poi_ori_test,test_non_target,best_noise.cpu()*multi_test,lab,None) # change original labels of poisoned inputs to the target label
asr_loaders = torch.utils.data.DataLoader(test_non_target_change_image_label, batch_size=test_batch_size, shuffle=True, num_workers=2) # to computex the attack success rate (ASR)
print('Poison test dataset size is:',len(test_non_target_change_image_label))

Poison test dataset size is: 9000


In [72]:
#Clean acc test dataset
clean_test_loader = torch.utils.data.DataLoader(ori_test, batch_size=test_batch_size, shuffle=False, num_workers=2)

In [75]:
#Target clean test dataset
test_target = list(np.where(np.array(test_label)==lab)[0]) # grab test examples having label 2 (bird)
target_test_set = Subset(ori_test,test_target) # create a subset of target class test examples in order to compute Tar-ACC
target_test_loader = torch.utils.data.DataLoader(target_test_set, batch_size=test_batch_size, shuffle=True, num_workers=2) # to compute Tar-ACC

In [76]:
from util import AverageMeter
train_ACC = []
test_ACC = []
clean_ACC = [] # prediction accuracy on the clean test examples
target_ACC = []

In [80]:
model.train() # 50000 samples including 25 bird-class examples (poisoned samples)
for images, labels in clean_train_loader:
    images, labels = images.to(device), labels.to(device)
    model.zero_grad()
    optimizer.zero_grad()
    logits = model(images)
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    
    print('Logits:', logits.shape)
    _, predicted = torch.max(logits.data, 1)
    print('Predicted: ', predicted.shape)
    print("Labels:", labels.shape)
    break

Logits: torch.Size([150, 10])
Predicted:  torch.Size([150])
Labels: torch.Size([150])


In [53]:
for epoch in tqdm(range(training_epochs)):
    # Train
    model.train()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    pbar = tqdm(clean_train_loader, total=len(clean_train_loader)) # training dataset of the clean-label attack (contains some poisoned examples)
    for images, labels in pbar: # loop through each batch
        images, labels = images.to(device), labels.to(device)
        model.zero_grad()
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(logits.data, 1)
        acc = (predicted == labels).sum().item()/labels.size(0)
        acc_meter.update(acc)
        loss_meter.update(loss.item())
        pbar.set_description("Acc %.2f Loss: %.2f" % (acc_meter.avg*100, loss_meter.avg))
    train_ACC.append(acc_meter.avg)
    print('Train_loss:',loss)
    scheduler.step()
    
    # Testing attack effect
    model.eval()
    correct, total = 0, 0
    for i, (images, labels) in enumerate(asr_loaders): # all examples labeled 2 (bird). Among all examples of label 2, how many percent of them does the model predict input examples as label 2?
        # 9000 samples from test dataset (with labels changed to target label)
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            logits = model(images)
            out_loss = criterion(logits,labels)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    test_ACC.append(acc)
    print('\nAttack success rate %.2f' % (acc*100))
    print('Test_loss:',out_loss)
    
    correct_clean, total_clean = 0, 0
    for i, (images, labels) in enumerate(clean_test_loader): # original test CIFAR-10, no poisoned examples
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            logits = model(images)
            out_loss = criterion(logits,labels)
            _, predicted = torch.max(logits.data, 1)
            total_clean += labels.size(0)
            correct_clean += (predicted == labels).sum().item()
    acc_clean = correct_clean / total_clean
    clean_ACC.append(acc_clean)
    print('\nTest clean Accuracy %.2f' % (acc_clean*100))
    print('Test_loss:',out_loss)
    
    correct_tar, total_tar = 0, 0
    for i, (images, labels) in enumerate(target_test_loader): # compute Tar-ACC, meaning that computing prediction accuracy on a subset of examples labeled 2 from test CIFAR-10
        # 1000 samples labeled 2 (no poisoned) 
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            logits = model(images)
            out_loss = criterion(logits,labels)
            _, predicted = torch.max(logits.data, 1)
            total_tar += labels.size(0)
            correct_tar += (predicted == labels).sum().item()
    acc_tar = correct_tar / total_tar
    target_ACC.append(acc_tar)
    print('\nTarget test clean Accuracy %.2f' % (acc_tar*100))
    print('Test_loss:',out_loss)







































































































































































































































































































































































































































































































































































































































































































































































































































































Acc 32.94 Loss: 1.88: 100%|██████████| 334/334 [00:17<00:00, 18.58it/s]

Train_loss: tensor(1.4830, device='cuda:0', grad_fn=<NllLossBackward0>)



Attack success rate 8.81
Test_loss: tensor(2.8711, device='cuda:0')

Test clean Accuracy 46.40
Test_loss: tensor(1.5611, device='cuda:0')



Target test clean Accuracy 17.00
Test_loss: tensor(1.9129, device='cuda:0')








































































































































































































































































































































































































































































































































































































































































































































































































































































Acc 51.78 Loss: 1.32: 100%|██████████| 334/334 [00:10<00:00, 31.71it/s]

...

Train_loss: tensor(0.1827, device='cuda:0', grad_fn=<NllLossBackward0>)



Attack success rate 85.77
Test_loss: tensor(0.5963, device='cuda:0')

Test clean Accuracy 88.64
Test_loss: tensor(0.4308, device='cuda:0')



Target test clean Accuracy 92.90
Test_loss: tensor(0.1940, device='cuda:0')


























































































































































































































































































































































































































































































































































































































































































































































































 36%|███▌      | 71/200 [15:19<27:49, 12.95s/it]


KeyboardInterrupt: 

In [ ]:
#ours -- higher_configureations
from matplotlib import pyplot as plt
half = np.arange(0,training_epochs)
plt.figure(figsize=(12.5,8))
plt.plot(half, np.asarray(train_ACC)[half], label='Training ACC', linestyle="-.", marker="o", linewidth=3.0, markersize = 8)
plt.plot(half, np.asarray(test_ACC)[half], label='Attack success rate', linestyle="-.", marker="o", linewidth=3.0, markersize = 8) # ASR
plt.plot(half, np.asarray(clean_ACC)[half], label='Clean test ACC', linestyle="-.", marker="o", linewidth=3.0, markersize = 8) # ACC
plt.plot(half, np.asarray(target_ACC)[half], label='Target class clean test ACC', linestyle="-", marker="o", linewidth=3.0, markersize = 8) # Tar-ACC
# plt.plot(half, np.asarray(test_unl_ACC)[half], label='protected test ACC', linestyle="-.", marker="o", linewidth=3.0, markersize = 8)
plt.ylabel('ACC', fontsize=24)
plt.xticks(fontsize=20)
plt.xlabel('Epoches', fontsize=24)
plt.yticks(np.arange(0,1.1, 0.1),fontsize=20)
plt.legend(fontsize=20,bbox_to_anchor=(1.016, 1.2),ncol=2)
plt.grid(color="gray", linestyle="-")
plt.show()

dis_idx = clean_ACC.index(max(clean_ACC))
print(train_ACC[dis_idx])
print('attack',test_ACC[dis_idx])
print(clean_ACC.index(max(clean_ACC)))
print('all class clean', clean_ACC[dis_idx])
print('target clean',target_ACC[dis_idx])